In [7]:
import numpy as np
import os
import glob
import logging
import json
import sys
sys.path.append("/workspace/zecheng/modelzipper/projects")
from tqdm import tqdm
from concurrent import futures
from argparse import ArgumentParser
from change_deepsvg.svglib.svg import SVG
from change_deepsvg.svglib.geom import Bbox, Angle, Point
from change_deepsvg.difflib.tensor import SVGTensor
from modelzipper.tutils import *
import torch
from tqdm import trange
from PIL import Image

content = auto_read_data("/zecheng2/vqllama/test_vqllama_quantizer/test_12/epoch_37/compress_level_1_predictions.pkl")

def sanint_check_svg_tensor(x):
    """
    x: batch_size x seq_len x (7, 9)
    """
    if x.ndim == 2:
        x = x.unsqueeze(0)
    if x.size(-1) == 9:
        x[:, :, 0][x[:, :, 0] == 100] = 1
        x[:, :, 0][x[:, :, 0] == 200] = 2
    elif x.size(-1) == 7:
        # add two columns
        x_0_y_0 = torch.zeros((x.size(0), x.size(1), 2), dtype=x.dtype, device=x.device)
        x_0_y_0[:, 1:, 0] = x[:, :-1, -2]  # x_3 of the previous row
        x_0_y_0[:, 1:, 1] = x[:, :-1, -1]  # y_3 of the previous row
        x = torch.cat((x[:, :, :1], x_0_y_0, x[:, :, 1:]), dim=2)
    return x

def convert_svg(t, colored=False):
    svg = SVGTensor.from_data(t)
    svg = SVG.from_tensor(svg.data, viewbox=Bbox(200))
    svg.numericalize(n=200)
    if colored:
        import pdb; pdb.set_trace()
        svg = svg.normalize().split_paths().set_color("random")
    str_svg = svg.to_str()
    return svg, str_svg

begin to read data from /zecheng2/vqllama/test_vqllama_quantizer/test_12/epoch_37/compress_level_1_predictions.pkl ...


In [12]:
p_predict = content['p_predict']
raw_predict = content['raw_predict']
golden = content['golden']

sample_p_0 = p_predict[0]
sample_r_0 = raw_predict[0]
sample_g_0 = golden[0] 

sample_p_0 = sanint_check_svg_tensor(sample_p_0).squeeze(0)
print(sample_p_0.shape)


svg = SVGTensor.from_data(sample_p_0)
svg = SVG.from_tensor(svg.data, viewbox=Bbox(200))
svg.numericalize(n=200)

svg.normalize().split_paths().set_color("random")

torch.Size([308, 9])


SVG[Bbox(0.0 0.0 200.0 200.0)](
	SVGPathGroup(SVGPath(M[P(157.0, 100.0), P(156.0, 99.0)] L[P(156.0, 99.0), P(155.0, 192.0)] L[P(155.0, 192.0), P(48.0, 194.0)] L[P(48.0, 194.0), P(47.0, 12.0)]), SVGPath(M[P(47.0, 12.0), P(47.0, 11.0)] L[P(47.0, 11.0), P(160.0, 13.0)]), SVGPath(M[P(160.0, 13.0), P(158.0, 12.0)] L[P(158.0, 12.0), P(161.0, 105.0)] L[P(161.0, 105.0), P(161.0, 103.0)]), SVGPath(M[P(92.0, 17.0), P(91.0, 17.0)] C[P(91.0, 17.0), P(86.0, 22.0), P(86.0, 31.0), P(90.0, 41.0)]), SVGPath(M[P(90.0, 41.0), P(90.0, 40.0)] C[P(90.0, 40.0), P(94.0, 54.0), P(106.0, 69.0), P(115.0, 66.0)] C[P(115.0, 66.0), P(126.0, 59.0), P(122.0, 30.0), P(109.0, 21.0)] C[P(109.0, 21.0), P(102.0, 20.0), P(96.0, 19.0), P(92.0, 19.0)]), SVGPath(M[P(92.0, 19.0), P(93.0, 19.0)] L[P(93.0, 19.0), P(92.0, 19.0)]), SVGPath(M[P(113.0, 26.0), P(111.0, 27.0)] C[P(111.0, 27.0), P(125.0, 30.0), P(130.0, 41.0), P(124.0, 51.0)]), SVGPath(M[P(124.0, 51.0), P(125.0, 51.0)] C[P(125.0, 51.0), P(124.0, 64.0), P(114.0, 71.0), 